<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Arthropod_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Arthropod Taxonomy Orders Object Detection Dataset](https://www.kaggle.com/datasets/mistag/arthropod-taxonomy-orders-object-detection-dataset/)



###### Lepidoptera = butterfies and moths
###### Hymenoptera = wasps, bees and ants
###### Hemiptera = true bugs (cicadas, aphids, shield bugs, ...)
###### Odonata = dragonflies
###### Diptera = fies
###### Araneae = spiders
###### Coleoptera = beetles

###### NOT IN DATASET
###### Orthoptera = grasshoppers

In [1]:
import os
import re
import sys
import json
import time
import numpy as np
import pprint as pp
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib as mpl
import concurrent.futures
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt


AUTO = tf.data.AUTOTUNE
pp = pp.PrettyPrinter()

!pip install --quiet tf-models-official==2.9.2
sys.path.append('PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER')# load all the metadata

import tensorflow_models as tfm
from tensorflow_models.vision import box_ops as box

     |████████████████████████████████| 2.1 MB 4.2 MB/s 
     |████████████████████████████████| 4.6 MB 30.6 MB/s 
     |████████████████████████████████| 118 kB 64.7 MB/s 
     |████████████████████████████████| 662 kB 53.5 MB/s 
     |████████████████████████████████| 238 kB 53.2 MB/s 
     |████████████████████████████████| 1.3 MB 46.6 MB/s 
     |████████████████████████████████| 1.1 MB 47.7 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 352 kB 39.9 MB/s 


In [2]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/'
!kaggle datasets download -d mistag/arthropod-taxonomy-orders-object-detection-dataset
!unzip *.zip && rm *.zip

Streaming output truncated to the last 5000 lines.
  inflating: ArTaxOr/Lepidoptera/c9b07301ed0e.jpg  
  inflating: ArTaxOr/Lepidoptera/c9d742746b8a.jpg  
  inflating: ArTaxOr/Lepidoptera/c9d78061b77d.jpg  
  inflating: ArTaxOr/Lepidoptera/ca2ff8921244.jpg  
  inflating: ArTaxOr/Lepidoptera/ca338092c0ec.jpg  
  inflating: ArTaxOr/Lepidoptera/ca35e5c89d94.jpg  
  inflating: ArTaxOr/Lepidoptera/ca4cb66fcd4a.jpg  
  inflating: ArTaxOr/Lepidoptera/ca5e1fe4cbc0.jpg  
  inflating: ArTaxOr/Lepidoptera/ca5e2a4139c0.jpg  
  inflating: ArTaxOr/Lepidoptera/ca63af0a9774.jpg  
  inflating: ArTaxOr/Lepidoptera/ca652f69b2e7.jpg  
  inflating: ArTaxOr/Lepidoptera/ca95914f205e.jpg  
  inflating: ArTaxOr/Lepidoptera/ca997241704b.jpg  
  inflating: ArTaxOr/Lepidoptera/cb0ad7fb32fa.jpg  
  inflating: ArTaxOr/Lepidoptera/cb0aec5a6b79.jpg  
  inflating: ArTaxOr/Lepidoptera/cb134256077c.jpg  
  inflating: ArTaxOr/Lepidoptera/cb22cb2317b4.jpg  
  inflating: ArTaxOr/Lepidoptera/cb35eb6c2e67.jpg  
  inflating: 

# [Data Exploring](https://www.kaggle.com/code/mistag/starter-arthropod-taxonomy-orders-data-exploring)

The Arthropod Taxonomy Orders dataset is a collection of highres images annotated with labels from the taxanomy rank order. Annotations have been made with VoTT. VoTT stores all metadata in json files. In this kernel we will import all the metadata into DataFrames and export metadata to a variety of formats for object detection model training.
The dataset is distributed under CC BY-NC-SA 4.0

In [3]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import glob

pfiles = glob.glob('/content/ArTaxOr/**/*.vott', recursive=True)

df = pd.DataFrame()
for p in pfiles:
  print(p)
  with open(p) as f:
    pdata = json.load(f)
    print(pdata)
    df = df.append(pd.DataFrame(list(pdata['assets'].values())), ignore_index=True)

df['path'] = df['path'].str.replace('file:F:/', '')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
df

{'name': 'Odonata', 'securityToken': 'Diptera Token', 'sourceConnection': {'name': 'Odonata', 'providerType': 'localFileSystemProxy', 'providerOptions': {'encrypted': 'eyJjaXBoZXJ0ZXh0IjoiODY0NWJmNDA1NDNjOGVlYmZhZDQzZTliYzU4NjM0MjliZDcyNjRlODJhYzRkODNhNGFhMzhiY2U2N2UyYmFjOTU3MmY4NWEzYWVkYjI0ODAwODU3MGExOWUyMzRmZWQ4IiwiaXYiOiI1ZDkzYjc2MWI5ZTg2MzUxOWVlYzlkYmIzZjNkMTA3ODUyZmIyOTlmY2ZmZWExYWMifQ=='}, 'id': 'Ko1s85UOf'}, 'targetConnection': {'name': 'Odonata annotations', 'providerType': 'localFileSystemProxy', 'providerOptions': {'encrypted': 'eyJjaXBoZXJ0ZXh0IjoiOGVkNTIwMDU5OGM0NWE1MDY2MzgwMTE1MTkxMGMyNDcxNjI4ZWRjOGQ4M2EwMzQ3OTU0NmU3NmE0MDY4MTZmNGNhNDAzODFhMjliNWJlODM3NzQ0MjEyOTdjZjc4NmNhODhhZWQzNDdhZGRiZmZiNTIwODY2MDRmN2Y0ZTczNTUiLCJpdiI6IjUyYTgzMTBlOWRlNWViNjVjMzU5ODFiYWFjYmMxYWMwYzE3NzMzMGQ1MTU2MmNjMyJ9'}, 'id': 'csXqIsO36'}, 'videoSettings': {'frameExtractionRate': 15}, 'tags': [{'name': 'Lepidoptera', 'color': '#5db300'}, {'name': 'Coleoptera', 'color': '#e81123'}, {'name': 'Hymenopt

,format,id,name,path,size,state,type
0,jpg,9c8f23b82ee4a2d682291faa7a56cbbf,00026d79d067.jpg,ArTaxOr/Lepidoptera/00026d79d067.jpg,"{'width': 3888, 'height': 2592}",2,1
1,jpg,9047380761065400114098ba4a37eb4c,00028c56bf09.jpg,ArTaxOr/Lepidoptera/00028c56bf09.jpg,"{'width': 5184, 'height': 3456}",2,1
2,jpg,511bf35dd66f49103ff9ee830d6022e0,00126d9a3bea.jpg,ArTaxOr/Lepidoptera/00126d9a3bea.jpg,"{'width': 4032, 'height': 3024}",2,1
3,jpg,518d6bc681b944adb5fbfecb2e78ac13,002b37ac08e1.jpg,ArTaxOr/Lepidoptera/002b37ac08e1.jpg,"{'width': 1920, 'height': 1521}",2,1
4,jpg,4338bc92a1392d3d5a61ed0c90a56be4,0032708c2542.jpg,ArTaxOr/Lepidoptera/0032708c2542.jpg,"{'width': 4608, 'height': 3456}",2,1
...,...,...,...,...,...,...,...
15371,jpg,27c45505c9d0f024745b6cad9b232d6b,ff926865268a.jpg,ArTaxOr/Odonata/ff926865268a.jpg,"{'width': 2000, 'height': 1537}",2,1
15372,jpg,c6e4c8624ca87afdfd590f04ebcc6871,ffab00fe2499.jpg,ArTaxOr/Odonata/ffab00fe2499.jpg,"{'width': 2048, 'height': 1329}",2,1
15373,jpg,25569532ac3f65cd68c8ad53ceaead14,ffc1fd21a8bb.jpg,ArTaxOr/Odonata/ffc1fd21a8bb.jpg,"{'width': 2048, 'height': 1407}",2,1
15374,jpg,ff13dae44aead408c5a11f6063e29075,ffc4adb8b04e.jpg,ArTaxOr/Odonata/ffc4adb8b04e.jpg,"{'width': 1384, 'height': 1038}",2,1


In [6]:
labels = pd.DataFrame(list(pdata['tags']))[:-3]
labels

,name,color
0,Lepidoptera,#5db300
1,Coleoptera,#e81123
2,Hymenoptera,#6917aa
3,Diptera,#015cda
4,Araneae,#4894fe
5,Hemiptera,#257ffe
6,Odonata,#257ffe


In [7]:
p = 'ArTaxOr/Diptera/fff86b0ae807.jpg'
index = p[::-1].find('/')+1
p[:-index]

'ArTaxOr/Diptera'

In [8]:
anno=pd.DataFrame(columns=['label', 'label_idx', 'xres', 'yres', 'height', 'width', 'left', 'top', 
                           'right', 'bottom', 'area', 'xcenter', 'ycenter', 'blurred',
                           'occluded', 'truncated', 'file', 'id'])


#get all the image's annotation details (object data) from a json file and store it in a dataframe
for i in range(len(df)):  #15376 images(with details)
  p = df['path'][i]
  index = p[::-1].find('/')+1
  json_file=f'/content/{p[:-index]}/annotations/{df["id"][i]}-asset.json'

  if os.path.isfile(json_file):
    with open(json_file) as f:
      adata = json.load(f)
    xres=adata['asset']['size']['width']
    yres=adata['asset']['size']['height'] 
    for j in range(len(adata['regions'])):
            h=adata['regions'][j]['boundingBox']['height']/yres
            w=adata['regions'][j]['boundingBox']['width']/xres
            tags=adata['regions'][j]['tags']
            anno=anno.append({'label': tags[0],
                              'label_idx': labels[labels.name==tags[0]].index[0],
                              'xres': xres,
                              'yres': yres,
                              'height': h,
                              'width': w,                              
                              'left': adata['regions'][j]['boundingBox']['left']/xres,
                              'top': adata['regions'][j]['boundingBox']['top']/yres,
                              'right': adata['regions'][j]['boundingBox']['left']/xres+w,
                              'bottom': adata['regions'][j]['boundingBox']['top']/yres+h, 
                              'area': h*w,
                              'xcenter': adata['regions'][j]['boundingBox']['left']/xres+0.5*w,
                              'ycenter': adata['regions'][j]['boundingBox']['top']/yres+0.5*h,
                              'blurred': int(any(ele == '_blurred' for ele in tags)),
                              'occluded': int(any(ele == '_occluded' for ele in tags)),
                              'truncated': int(any(ele == '_truncated' for ele in tags)),
                              'file': adata['asset']['path'].replace('file:F:/',''),
                              'id': adata['asset']['id'],}, ignore_index=True)

In [9]:
anno.sample(5)

,label,label_idx,xres,yres,height,width,left,top,right,bottom,area,xcenter,ycenter,blurred,occluded,truncated,file,id
12468,Araneae,4,2048,1540,0.502874,0.278098,0.322553,0.253831,0.600651,0.756705,0.139848,0.461602,0.505268,1,0,0,ArTaxOr/Araneae/7d8ec1496dce.jpg,f3398e08f294ace7f51ae72e5a00039f
18083,Odonata,6,2048,1365,0.343458,0.349948,0.345405,0.318536,0.695353,0.661994,0.120192,0.520379,0.490265,0,0,0,ArTaxOr/Odonata/97bfe0129ad5.jpg,8a063b9a668602fb472e6007d6ee7081
4027,Hemiptera,5,1869,1559,0.721264,0.221246,0.386432,0.119732,0.607678,0.840996,0.159577,0.497055,0.480364,0,0,0,ArTaxOr/Hemiptera/4c70925b1ab9.jpg,b089261ed0a67a4de75bacd44fb05d8d
11421,Araneae,4,1037,718,0.817910,0.621640,0.181943,0.078933,0.803584,0.896844,0.508446,0.492764,0.487889,0,0,0,ArTaxOr/Araneae/08522225994e.jpg,12b5b9b06fbc372dae4ca3f4b90600c8
4528,Hemiptera,5,2048,1536,0.784483,0.520833,0.239943,0.093870,0.760776,0.878352,0.408585,0.500359,0.486111,0,0,0,ArTaxOr/Hemiptera/7596c8a9709e.jpg,ea79821734ac9b1441179e105d4429d6


In [ ]:
# sns.relplot(x="width", y="height", hue="label", col="label", data=anno)

In [ ]:
# sns.jointplot(x='width', y='height', data=anno.loc[anno['label'] == 'Diptera'])

In [ ]:
sns.set(rc={'figure.figsize': (12,6)})
# sns.violinplot(x=anno['label'], y=anno['area'])

In [ ]:
# graph=sns.countplot(data=anno, x='label')
# graph.set_xticklabels(graph.get_xticklabels(), rotation=90)

# for p in graph.patches:
#   height = p.get_height()
#   graph.text(p.get_x() + p.get_width()/2, height + 0.1, height, ha='center')

In [ ]:
temp = 'truncated'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
# sns.countplot(x=temp, hue='label', data=df2)

In [ ]:
temp = 'blurred'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
# sns.countplot(x=temp, hue='label', data=df2)

In [ ]:
temp = 'occluded'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
# sns.countplot(x=temp, hue='label', data=df2)

## Image exploration


In [17]:
def attribution(fname):
    img = Image.open(fname)
    exif_data = img._getexif()
    img.close()
    if len(exif_data[315]) > 0:
        s='Photo: '+exif_data[315]
    else:
        s=exif_data[37510][8:].decode('ascii')
    return s

def plot_img(axes, image_df, highlight=True):
    image_file_name='/content/'+image_df.iloc[0].file
    im = Image.open(image_file_name)
    im.thumbnail((300,300),Image.ANTIALIAS)  #A small image representation of a larger image
    draw = ImageDraw.Draw(im)
    xres, yres = im.size[0], im.size[1] # X, Y Resolution
    for i in range(len(image_df)):
        selected_img = image_df.iloc[i]
        if highlight==True:
            color=(255, 0, 0) if i == 0 else (128, 128, 128)          
        else:
            color=labels[labels.name == selected_img.label].color.iloc[0]
        
        #draw a boundingbox
        draw.rectangle([int(selected_img['left']*xres),
                        int(selected_img['top']*yres),
                        int(selected_img['right']*xres),
                        int(selected_img['bottom']*yres)], outline=color, width=2)
        
    plt.setp(axes, xticks=[], yticks=[])  #remove the grids
    axes.set_title(image_df.iloc[0].label+'\n'+attribution(image_file_name))
    plt.imshow(im)

In [ ]:
fig = plt.figure(figsize=(16,26))
for i in range(len(labels)):
  for j in range(3):
    #Select 3 images with the largest area
    # ldf=anno[anno.label == labels.name[i]].nlargest(3, 'area') 

    #Select images with the most objects
    a=anno[anno.label == labels.name[i]]['id'].value_counts()
    ldf=anno[anno.id == a.index[j]]
    axes = fig.add_subplot(len(labels), 3, 1+i*3+j)

    
    # plot_img(axes, anno[anno.id == ldf.iloc[j].id].sort_values(by=['area'], ascending=False), highlight=False)

## Metadata export

export metadata to various formats for object detection model training


### CSV

In [19]:
header = ['file', 'label', 'height', 'width', 'left', 'top', 'right', 'bottom'] # change as required
anno.to_csv('./ArTaxOr.csv', index=False, columns = header) 

### Pascal VOC

Pascal VOC files are xml format, and there is one xml file per image file, with same name.

In [20]:
!{sys.executable} -m pip install pascal_voc_writer
from pascal_voc_writer import Writer

if not os.path.exists('voc'):
    os.mkdir('voc')

for i in range(len(df)):
    ldf=anno[anno.id == df.id[i]].reset_index()
    p=df.path[i].split('/')
    image_xml = p[2].replace('.jpg','.xml') 
    width, height = ldf.xres[0], ldf.yres[0]
    
    writer = Writer(df.path[i], width, height)
    for j in range(len(ldf)):
        writer.addObject(ldf.label[j], 
                         int(ldf.left[j]*width), 
                         int(ldf.top[j]*height), 
                         int(ldf.right[j]*width),
                         int(ldf.bottom[j]*height))
    writer.save(f'./voc/{image_xml}')
print(os.listdir("./voc"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['fdda8ecab65a.xml', '10f22c23d9f2.xml', '71e42bfb87dd.xml', '9627d904691b.xml', '6435354db11e.xml', '8dbe089e7491.xml', '6d17a4606e21.xml', 'da9cdc6e635c.xml', 'd73dd5b16c79.xml', '79dc02c5f6d6.xml', '290ad1c1e331.xml', '88e6c6897556.xml', 'a3501a06ee6a.xml', '9786b943cab4.xml', 'ea0be0adc25b.xml', '049e4b1e547a.xml', '22c0cc203d0b.xml', '7188c0cc8c9d.xml', '4edac66b62af.xml', 'a058e4cf5442.xml', '12b127e36a18.xml', 'e8cbba83803c.xml', '0995407bb915.xml', '14253d0a52dd.xml', 'ab187250dee3.xml', '9d88e7ccc38e.xml', '696f2231bcfc.xml', '61b03e157113.xml', '238cc6edb6db.xml', 'fe606aca80be.xml', '12466c8f9222.xml', '6238cc39e8ce.xml', 'e7c8a1006595.xml', 'ffd97e78e0f4.xml', 'a3fcd315f8a1.xml', '4eb35f0d957f.xml', 'd751eae2ac32.xml', 'faa668c22ba6.xml', 'f044dbed88b1.xml', 'ec7798745955.xml', '08e4fc3e2eaf.xml', '1e7671ce8810.xml', '73924f6f1827.xml', '80a2a81912b5.xml', '6a7baccd3b00.xml', 

### Darknet YOLOv3
Darknet expects one annotation file per image file. Each object is described by:

`class x_center y_center width height`

In [21]:
if not os.path.exists('labels'):
    os.mkdir('labels')

for i in range(len(df)):
    ldf=anno[anno.id == df.id[i]].reset_index()
    p=df.path[i].split('/') 
    image_txt = p[2].replace('.jpg','.txt') 
    
    file=open(f'./labels/{image_txt}','w')
    for j in range(len(ldf)):
        label_name=labels[labels.name == ldf.label[j]].index.to_list()
        file.write(f'{label_name[0]} {ldf.xcenter[j]} {ldf.ycenter[j]} {ldf.width[j]} {ldf.height[j]}\n')
    file.close()
print(os.listdir("./labels"))

['74d0deaa0e60.txt', 'd080b3bdd9d2.txt', '1f5826e35cf5.txt', 'a74c0593e7b0.txt', '950eae32a8e4.txt', '1dfd75560515.txt', 'b771ee0ab4c4.txt', 'c5bb76e4a6e5.txt', 'c9359c616bc5.txt', '8a034d342d43.txt', '79aa5cc42ebf.txt', '4bd772eb13b0.txt', '81f041a51b90.txt', '875890b81649.txt', 'a7ee5e90585a.txt', '412ca6ddbe8c.txt', 'dc7e1179a913.txt', '522a0734571c.txt', '5d13dc834313.txt', '1829ae48ae3f.txt', 'fe0d0d4ba2c3.txt', '4c5f946fe00e.txt', '749a2847f212.txt', '9c911b78b009.txt', 'a7f5ad625414.txt', 'edd74f65df12.txt', 'a5dcd1612b3e.txt', '3369271558a3.txt', 'e23416566335.txt', 'cf55106cec96.txt', 'abcd22e9aa43.txt', '0c5a6cb4a8e9.txt', 'b9d2c90d5eac.txt', 'aab042e0f747.txt', 'bb07702f63ee.txt', '8dfb5e949da5.txt', 'a5356d2a3c03.txt', 'c1a53817ea47.txt', '6167700b8203.txt', 'd4b356f36006.txt', 'd269ce938903.txt', '603b9df0f10b.txt', '16b9585816e3.txt', '17c8e526fa2f.txt', '4518d1bb44b1.txt', '36548c67f7fd.txt', '6660425a079b.txt', '4151c8f39452.txt', 'f8f2bc16d5ed.txt', '4edec9dbefb4.txt',

### Pickle

In [22]:
labels.to_pickle('./ArTaxOr_labels.pkl')
df.to_pickle('./ArTaxOr_filelist.pkl')
anno.to_pickle('./ArTaxOr_objects.pkl')

### TensorFlow TFRecords

#### Protocol buffers
TFRecords are Protocol buffers. Protocol buffers are a way of serializing structured data in a compact and efficient way. A Protocol buffer message is defined by a .proto file. Example:

```
syntax = "proto3"; // define protocol version

// Define "Employee" message with 3 fields
message Employee {
 string name = 1; // a unique number is assigned to each field
 int32 company_id = 2;
 string address = 3;
}

```


A single .proto-file can define multiple messages and messages can be nested. The beauty of protocol buffers is that the .proto-files can be compiled into a library (language of choice) with access functions using the protoc compiler. But for TFRecords the access functions we need are already part of TensorFlow, so no need to worry about compiling .proto files.

#### TFRecord format with simple data
TFRecords are made to support just about any type of data, and that means nesting basic features into a hierarchy of features.TFRecords supports only three different data types:

bytes
float
int64

So any data must be converted into one of these three types.

In [23]:
# tf.train.Feature can be used to convert data into lists of these types.
int_list = tf.train.Feature(int64_list=tf.train.Int64List(value=np.arange(8)))
int_list

int64_list {
  value: 0
  value: 1
  value: 2
  value: 3
  value: 4
  value: 5
  value: 6
  value: 7
}

In [24]:
float_list = tf.train.Feature(float_list=tf.train.FloatList(value=np.random.rand(10)))
float_list

float_list {
  value: 0.5189214944839478
  value: 0.33266153931617737
  value: 0.0643123909831047
  value: 0.538964569568634
  value: 0.31707319617271423
  value: 0.08194542676210403
  value: 0.4263155460357666
  value: 0.8758863806724548
  value: 0.6110150814056396
  value: 0.14489707350730896
}

In [25]:
bytes_list = tf.train.Feature(bytes_list=tf.train.BytesList(value=['hello, my name is Esmail'.encode()]))
bytes_list

bytes_list {
  value: "hello, my name is Esmail"
}

In [26]:
features = tf.train.Features(feature={
    'integers': int_list,
    'float': float_list,
    'description': bytes_list
})
features

feature {
  key: "description"
  value {
    bytes_list {
      value: "hello, my name is Esmail"
    }
  }
}
feature {
  key: "float"
  value {
    float_list {
      value: 0.5189214944839478
      value: 0.33266153931617737
      value: 0.0643123909831047
      value: 0.538964569568634
      value: 0.31707319617271423
      value: 0.08194542676210403
      value: 0.4263155460357666
      value: 0.8758863806724548
      value: 0.6110150814056396
      value: 0.14489707350730896
    }
  }
}
feature {
  key: "integers"
  value {
    int64_list {
      value: 0
      value: 1
      value: 2
      value: 3
      value: 4
      value: 5
      value: 6
      value: 7
    }
  }
}

The last step is to create an example TFRecord. So, example is not a very good name, but that is what the TFRecord is called. We simply pass the list of features to `tf.train.Example.`

In [27]:
tf_record = tf.train.Example(features=features)
tf_record

features {
  feature {
    key: "description"
    value {
      bytes_list {
        value: "hello, my name is Esmail"
      }
    }
  }
  feature {
    key: "float"
    value {
      float_list {
        value: 0.5189214944839478
        value: 0.33266153931617737
        value: 0.0643123909831047
        value: 0.538964569568634
        value: 0.31707319617271423
        value: 0.08194542676210403
        value: 0.4263155460357666
        value: 0.8758863806724548
        value: 0.6110150814056396
        value: 0.14489707350730896
      }
    }
  }
  feature {
    key: "integers"
    value {
      int64_list {
        value: 0
        value: 1
        value: 2
        value: 3
        value: 4
        value: 5
        value: 6
        value: 7
      }
    }
  }
}

 write this record to a file using tf.io.TFRecordWriter

In [28]:
fname='example1.tfrecord'
with tf.io.TFRecordWriter(fname) as writer:
    writer.write(tf_record.SerializeToString())
print(f"Size of {fname} is {os.path.getsize(fname)} bytes")

Size of example1.tfrecord is 144 bytes


Next, read the file back in and access the data using tf.data.TFRecordDataset and tf.train.Example.FromString:

In [29]:
dataset = tf.data.TFRecordDataset(fname)
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())
parsed

features {
  feature {
    key: "description"
    value {
      bytes_list {
        value: "hello, my name is Esmail"
      }
    }
  }
  feature {
    key: "float"
    value {
      float_list {
        value: 0.5189214944839478
        value: 0.33266153931617737
        value: 0.0643123909831047
        value: 0.538964569568634
        value: 0.31707319617271423
        value: 0.08194542676210403
        value: 0.4263155460357666
        value: 0.8758863806724548
        value: 0.6110150814056396
        value: 0.14489707350730896
      }
    }
  }
  feature {
    key: "integers"
    value {
      int64_list {
        value: 0
        value: 1
        value: 2
        value: 3
        value: 4
        value: 5
        value: 6
        value: 7
      }
    }
  }
}

Access each feature using `parsed.features.feature['<feature name>']`



In [30]:
parsed.features.feature['description'].bytes_list.value[0].decode()

'hello, my name is Esmail'

In [31]:
parsed.features.feature['integers'].int64_list.value[:]

[0, 1, 2, 3, 4, 5, 6, 7]

In [32]:
parsed.features.feature['float'].float_list.value[:]

[0.5189214944839478,
 0.33266153931617737,
 0.0643123909831047,
 0.538964569568634,
 0.31707319617271423,
 0.08194542676210403,
 0.4263155460357666,
 0.8758863806724548,
 0.6110150814056396,
 0.14489707350730896]

####TFRecords with image data

To make input function efficient, all preprocessing steps should be performed when creating TFRecords, otherwise the input function must repeat preprocessing each time the training passed over the TFRecord. Typical preprocessing steps are:

##### Decode image from jpg etc.
##### Resize image
##### Convert to float
##### Normalize to [0,1] range

Augumentation is not something to do in a TFRecord, since agumentation could be different each time training passes over the data

In [33]:
import hashlib
from io import BytesIO
from PIL import Image, ImageFont, ImageDraw
ARTAXOR_PATH = '/content/'

pickles='/content/'
objectdf=pd.read_pickle(pickles+'ArTaxOr_objects.pkl')
labels=pd.read_pickle(pickles+'ArTaxOr_labels.pkl')
objectdf.sample(5)

,label,label_idx,xres,yres,height,width,left,top,right,bottom,area,xcenter,ycenter,blurred,occluded,truncated,file,id
5599,Hemiptera,5,777,1203,0.856322,0.713650,0.112991,0.045977,0.826641,0.902299,0.611114,0.469816,0.474138,0,0,0,ArTaxOr/Hemiptera/d14d14d7243a.jpg,e8837e7845c3542b32f3eeefa4945edd
13558,Araneae,4,800,615,0.725096,0.751841,0.186326,0.160920,0.938167,0.886015,0.545157,0.562247,0.523467,0,0,0,ArTaxOr/Araneae/f080d040ef24.jpg,84e771d3321c01bb62fced31ee4ead7d
7378,Coleoptera,1,5184,3456,0.245088,0.226740,0.411440,0.376083,0.638181,0.621171,0.055571,0.524810,0.498627,0,0,0,ArTaxOr/Coleoptera/751de26a29a3.jpg,04a8089e7096a53552a3d69f72caa147
468,Lepidoptera,0,1920,1280,0.158221,0.191592,0.399035,0.452608,0.590627,0.610829,0.030314,0.494831,0.531719,0,1,0,ArTaxOr/Lepidoptera/267112a42cf6.jpg,72c73e335a1d7cc72faeef858974453e
10664,Diptera,3,1518,1168,0.584291,0.633014,0.191542,0.204981,0.824556,0.789272,0.369865,0.508049,0.497126,0,0,0,ArTaxOr/Diptera/bdb0c15b4055.jpg,1a83c0ff621755d1adc830f408edc275


Define a function that creates a tf.train.Example from an image and the objects contained within.

    Note that TensorFlow Object Detection API expects jpeg-encoded image data

, so the only preprocessing to be done is resize (optional) before the image is re-encoded (via a memory buffer using BytesIO).


The input to this function is a DataFrame that contains one row per object and the columns shown above. This record is generous with features, but that is to make sure that the TF Object Detection API will successfully run the evaluation phase during training.

In [34]:
# Fetch attribution string from image EXIF data
def get_attribution(file):
    with Image.open(file) as img:
        exif_data = img._getexif()
    s='Photo: unknown'
    if exif_data is not None:
        if 37510 in exif_data:
            if len(exif_data[37510]) > 0:
                s = exif_data[37510][8:].decode('ascii')
        if 315 in exif_data:
            if len(exif_data[315]) > 0:
                s = 'Photo: ' + exif_data[315]
    return s

# Create example for TensorFlow Object Detection API
def create_tf_example(imagedf, longest_edge=1024):  
    fname = ARTAXOR_PATH+imagedf.file.iloc[0]
    filename=fname.split('/')[-1] # exclude path
    by = get_attribution(fname)
    img = Image.open(fname, "r")
    # resize image if larger that longest edge while keeping aspect ratio
    if max(img.size) > longest_edge:
        img.thumbnail((longest_edge, longest_edge), Image.ANTIALIAS)
    height = img.size[1] # Image height
    width = img.size[0] # Image width
    buf= BytesIO()
    img.save(buf, format= 'JPEG') # encode to jpeg in memory
    encoded_image_data= buf.getvalue()
    image_format = b'jpeg'
    source_id = filename.split('.')[0]
    license = 'CC BY-NC-SA 4.0'
    # A hash of the image is used in some frameworks
    key = hashlib.sha256(encoded_image_data).hexdigest()   
    # object bounding boxes 
    xmins = imagedf.left.values # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = imagedf.right.values # List of normalized right x coordinates in bounding box
    ymins = imagedf.top.values # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = imagedf.bottom.values # List of normalized bottom y coordinates in bounding box
    # List of string class name & id of bounding box (1 per box)
    object_cnt = len(imagedf)
    classes_text = []
    classes = []
    for i in range(object_cnt):
        classes_text.append(imagedf.label.iloc[i].encode())
        classes.append(1+imagedf.label_idx.iloc[i])
    # unused features from Open Image 
    depiction = np.zeros(object_cnt, dtype=int)
    group_of = np.zeros(object_cnt, dtype=int)
    occluded = imagedf.occluded.values #also Pascal VOC
    truncated = imagedf.truncated.values # also Pascal VOC
    # Pascal VOC
    view_text = []
    for i in range(object_cnt):
        view_text.append('frontal'.encode())
    difficult = np.zeros(object_cnt, dtype=int)

    tf_record = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[source_id.encode()])),
        'image/license': tf.train.Feature(bytes_list=tf.train.BytesList(value=[license.encode()])),
        'image/by': tf.train.Feature(bytes_list=tf.train.BytesList(value=[by.encode()])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image_data])),
        'image/key/sha256': tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode()])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_format])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
        'image/object/depiction': tf.train.Feature(int64_list=tf.train.Int64List(value=depiction)),
        'image/object/group_of': tf.train.Feature(int64_list=tf.train.Int64List(value=group_of)),
        'image/object/occluded': tf.train.Feature(int64_list=tf.train.Int64List(value=occluded)),
        'image/object/truncated': tf.train.Feature(int64_list=tf.train.Int64List(value=truncated)),
        'image/object/difficult': tf.train.Feature(int64_list=tf.train.Int64List(value=difficult)),
        'image/object/view': tf.train.Feature(bytes_list=tf.train.BytesList(value=view_text))
    }))
    return tf_record

In [ ]:
#Example
sample_file='ArTaxOr/Hemiptera/00600a41bcaf.jpg'
imagedf=objectdf[objectdf.file == sample_file]
tfr=create_tf_example(imagedf)
# print(tfr)
fname='./image_ex1.tfrecord'
with tf.io.TFRecordWriter(fname) as writer:
    writer.write(tfr.SerializeToString())
print("Size of {} is {}kbytes".format(fname, os.path.getsize(fname)//1024))